# Pandas: Tópicos extra

En este anexo se revisan algunos tópicos específicos relacionados a la librería `pandas` que no fueron cubiertos anteriormente, estos son

- Objeto `pandas.Series`
- Gráficos a partir de objetos de pandas
- Comunicación con bases de datos SQL
- Guardar y leer datos en formato JSON y HDF5

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import YouTubeVideo
from functools import partial
YouTubeVideo_formato = partial(YouTubeVideo, modestbranding=1, disablekb=0,
                               width=640, height=360, autoplay=0, rel=0, showinfo=0)

## Objeto `pandas.Series`

El objeto [`pandas.Series`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) es  un arreglo de una dimensión (vector) que **representa una secuencia** 

- Los elementos de la secuencia se identifican con un índice etiquetado `index`
- Todos los elementos son de un mismo tipo `dtype`
- La serie se identifica con un nombre `name`

A continuación veremos algunas formas de crear `Series`

**Construyendo un objeto `Series` a partir de un dataframe**

Cuando pedimos **una columna** de un DataFrame el objeto retornado es de tipo `Series`

Tecnicamente, **una fila** de un DataFrame también retorna como `Series` sin embargo los tipos se mezclan

In [ ]:
clientes = ['Pablo', 'Marianna', 'Matthieu', 'Luis', 'Eliana', 'Cristobal']

ventas = {
    'lechugas [unidades]': [1, 0, 1, 2, 0, 0],
    'papas [kilos]': [0.5, 2, 1.5, 1.2, 0, 5]
}

df = pd.DataFrame(data=ventas, index=clientes)

In [ ]:
display(f'La columna de lechugas es un objeto {type(df["lechugas [unidades]"])}',
        f'cuyo tipo es {df["lechugas [unidades]"].dtype}',
        f'La fila Matthieu es un objeto {type(df.loc["Matthieu"])}',
        f'cuyo tipo es {df.loc["Matthieu"].dtype}') 

**Construyendo un objeto `Series` a partir de otras estructuras de datos**

Un objeto `Series` se puede crear de forma más general usando el constructor

```python
pandas.Series(data=None, 
              index=None,
              dtype=None, 
              name=None, 
              copy=False, 
              fastpath=False)

```

donde `data` puede ser un  diccionarios, una lista o un ndarray

Por ejemplo:

In [ ]:
plan_diario= {'dormir': 7, 'comer': 1, 'quehaceres': 1, 'trabajo': 10, 'procastinar': 5}

pd.Series(plan_diario, name='mi planificación de hoy')

Notas:

- Una columna o una fila de un `DataFrame` es un `Series`
- Varias `Series` se pueden unir para formar un `DataFrame`

## Gráfico a partir de DataFrames

Se pueden crear gráficos sencillos directamente de un `DataFrame`

Puedes revisar en detalle la API para graficar en este [link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4), tight_layout=True)
df.plot(ax=ax, kind='line', subplots=True);

La API es útil para hacer gráficos rapidamente. Si necesitamos mayor flexibilidad que la que ofrece la API siempre podemos extraer la data y graficarla con `matplotlib` o `seaborn`

## Lectura de bases de datos SQL


Pandas es capaz de conectar y hacer consultas en lenguaje SQL a una base de datos externa y retornar el resultado como un DataFrame usando la función [`read_sql_query`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) 

```python
pd.read_sql_query(sql, # Consulta SQL en formato string
                  con, # dirección a la base de datos o objeto de conexión
                  index_col=None, # Selecciona la columna que actuara como índice del DataFrame
                  parse_dates=None, # Igual que read_csv y read_excel
                  ...
                 )
```

También se puede usar el atributo

```python
df.to_sql(name, # string: el nombre de la tabla
          con, # Engine con conexión
          if_exists: str = 'fail', # Que hacer si la tabla ya existe: fail, replace, append
          index: bool = True, # Escribir el índice del dataframe como columna
          ...
         )
```

**Qué es SQL?**

Structured Query Languaje (SQL) es un lenguaje estándar ampliamente usado para consultar, crear, modificar y eliminar bases de datos relacionales. 

**Qué es una base de datos relacional?**

Es un tipo de base de datos organizada como múltiples tablas. Por ejemplo


|id_cliente | nombre | apellido |
|----|----|----|
|1| Pablo | Huijse |
|2| Luis | Alvarez |
|3| Cristobal | Navarro |
|  | CLIENTES |  |

|id_orden | platanos | manzanas | id_cliente |
|----|----|----| ---- |
|1| 0 | 5 | 1 |
|2| 2 | 2 | 3 |
|3| 3 | 1 | 1 |
|  | ORDENES |  | | 

- Las filas se llaman entidades y las columnas atributos
- Cada tabla tiene una lalve primaria: id_orden e id_cliente
- La tabla ORDENES **está relacionada** a la tabla CLIENTES con la llave foranea: id_clientes
- Las tablas no pueden tener el mismo nombre 



**Donde corre la base de datos relacional?**

La base de datos relacional corre en un sistema de manejo 

Algunos ejemplos populares son MySQL, PostgreSQL y SQLite3


**Ejemplo básico de una consulta SQL**

SQL es un lenguaje de alto nivel. Algunos comandos comunes son

- `SELECT`: recuperar un subconjunto de la tabla
- `INSERT`: insertar datos en una tabla
- `UPDATE`: actualizar datos en una tabla
- `DELETE`: eliminar datos de la tabla

La tabla que se quiere manipular se selecciona con el keyword `FROM`

Se agregan condiciones usando el keyword `WHERE`

Se puede usar `*` como alias para "todas las columnas"

Por ejemplo
```SQL
    SELECT A, B, C FROM mi_tabla WHERE D > 1
```

Esto recupera las valores de las columnas A, B y C que tegan un valor de la columna D mayor que 1 a partir de la tabla "mi_tabla" 

**Ejercicio 1: Crear una tabla SQL a partir de un DataFrame**

In [ ]:
# Descargamos los datos y creamos el dataframe
!wget -c http://www.censo2017.cl/wp-content/uploads/2017/12/Cantidad-de-Viviendas-por-Tipo.xlsx
df = pd.read_excel("Cantidad-de-Viviendas-por-Tipo.xlsx", 
                   sheet_name=1, # Importamos la segunda hoja (vivienda)
                   usecols=list(range(1, 20)), # Importamos las columnas 1 a 20
                   header=1, # El header está en la segunda fila
                   skiprows=[2], # Eliminamos la fila 2 ya que es invalida
                   index_col='ORDEN' # Usaremos el orden de las comunas como índice
                  ).dropna() # Eliminamos las filas con NaN

#Creamos una base de datos e insertartamos un DataFrame como tabla

import sqlite3  # SQLite3 es parte de la librería estándar de Python

# Creamos una base de datos persistente
with sqlite3.connect('censo.db') as conn:

    df.to_sql("censo_viviendas", # Insertamos una tabla llamada censo_viviendos
              conn, # Usamos el objeto conexión que acabos de crear
              if_exists='replace', 
              index=False)

**Ejercicio 2**

Escriba y pruebe una consulta SQL que recupere un `DataFrame` con las columnas "Viviendas Particulares Ocupadas con Moradores Presentes" y "NOMBRE COMUNA" para la provincia de Valdivia a partir de los datos del censo usados en la lección de "exploración y manipulación de datos con pandas"

Indicación: Cuando escriba su consulta encierre los nombres de las columnas con paréntesis cuadrados. Esto es necesario cuando los nombres contienen espacios en blanco

In [ ]:
# Solución: 

arg1 = "Viviendas Particulares Ocupadas con Moradores Presentes"
arg2 = "NOMBRE COMUNA"
arg3 = "NOMBRE PROVINCIA"
sql_string = f"SELECT [{arg1}], [{arg2}] FROM censo_viviendas WHERE [{arg3}] = 'VALDIVIA'"
display(sql_string)
with sqlite3.connect('censo.db') as conn:    
    df = pd.read_sql_query(sql_string, conn)
display(df)

**Comentarios y solución paso a paso**

In [ ]:
YouTubeVideo_formato('VsoZ1oh8Az8')

**Notas adicionales sobre SQLite**

- sqlite permite conectar a una base de datos local: RAM, disco, o disco externo montado
- sqlite no está diseñado para soportar múltiples usuarios conectados a una misma base de datos
- Otras alternativas: [SQL Alchemy](https://www.sqlalchemy.org/), [PostgreSQL+Python](http://initd.org/psycopg/), [Peewee](http://docs.peewee-orm.com/en/latest/)

## Guardar y leer una tabla en formato JSON

Podemos usar el atributo [`to_json`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html) para convertir un dataframe a este formato

```python
df.to_json(
    path_or_buf = None, # Ubicación en disco
    orient = None, # Indica el formato del string JSON
    ...
    )
```

Por ejemplo

```python
>>> df.to_json("pandas.json", orient='table')
```

crea un string pandas.json en el directorio actual

Luego la función `read_json`

```python
>>> df = pd.read_json("pandas.json", orient='table')
```

regenera el DataFrame que teniamos

## Guardar y leer una tabla en formato HDF5

Podemos usar el atributo `to_hdf` para convertir nuestra tabla a formato HDF5

```python
df.to_hdf(path_or_buf, # Path completo con nombre de archivo
          key: str, # Llave maestra del archivo
          mode: str = 'a', # Agrega lineas a un archivo existente (a) o crea una archivo nuevo (w)
          ...
         )
```

Por ejemplo

```python
df.to_hdf("pandas_hdf.h5", key='excel', mode='w')
```

crea un archivo pandas_hdf.h5 en el directorio actual

Para lectura podemos usar la función `read_hdf`

```python
mi_tabla_recuperada = pd.read_hdf("pandas_hdf.h5", key='/excel', mode='r')
```

```{note}
Para tener esta funcionalidad se requiere instalar Pytables mayor a 3.5: https://github.com/PyTables/PyTables/issues/719
```

Si necesita trabajar directamente sobre un archivo HDF5 en Python puede usar las librerías `PyTables` o `h5py`. A continuación se muestra un ejemplo con esta última:

```python
import h5py
with h5py.File("pandas_hdf.h5", mode="r") as f:
    print(f["excel"].keys())
    print(f["excel"]['block0_items'][()])
```